In [ ]:
import pandas as pd

df=pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

In [ ]:
df.info()

In [ ]:
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
        axis=1 , inplace =True)

In [ ]:
df

## EDA

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots

In [ ]:
a=pd.DataFrame(df['Attrition_Flag'].value_counts())

In [ ]:
fig = go.Figure(data=[go.Pie(labels=a.index, values=a.Attrition_Flag)])
fig.update_layout(title_text="Churned Customers")
fig.show()

### Income Category

In [ ]:
att_inc=df.groupby(['Attrition_Flag','Income_Category']).Income_Category.count().unstack()
print(att_inc)

In [ ]:
fig=att_inc.plot(kind='bar',title='Attrition Flag by Income Category',color = ['green','orange','red','blue','grey','purple'], alpha = .70)
fig.set_xlabel('Attrition Flag')
fig.set_ylabel('No. of Customers')
fig.legend(['$120K +','$40K - $60K','$60K - $80K','$80K - $120K','Less than $40K','Unknown'])
plt.show()

In [ ]:
att_cus = df.loc[df['Attrition_Flag']== 'Attrited Customer']
ext_cus = df.loc[df['Attrition_Flag']== 'Existing Customer']

In [ ]:
specs = [[{'type':'pie'}, {'type':'pie'}]]
fig = make_subplots(rows=1, cols=2, specs=specs,subplot_titles=['Attrited Customer','Existing Customer'])

In [ ]:
from collections import Counter
cnt = Counter(att_cus['Income_Category'])
perct = [(i, cnt[i] / len(att_cus['Income_Category']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
att_cus_inc = new.rename(columns={ 0 :'Income_Category', 1:'Percentage'})

In [ ]:
from collections import Counter
cnt = Counter(ext_cus['Income_Category'])
perct = [(i, cnt[i] / len(ext_cus['Income_Category']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
ext_cus_inc = new.rename(columns={ 0 :'Income_Category', 1:'Percentage'})

In [ ]:
fig.add_trace(go.Pie(labels=att_cus_inc['Income_Category'], values=att_cus_inc['Percentage'],name='No. of Customers'),1,1)
fig.add_trace(go.Pie(labels=ext_cus_inc['Income_Category'], values=ext_cus_inc['Percentage'],name='No. of Customers'),1,2)
fig=go.Figure(fig)
fig.update_layout(title_text='Income Category Distribution by Attrition')
fig.show()

### Education Level 

In [ ]:
att_edu=df.groupby(['Attrition_Flag','Education_Level']).Education_Level.count().unstack()
print(att_edu)

In [ ]:
fig=att_edu.plot(kind='bar',title='Attrition Flag by Education Level',color = ['green','orange','red','blue','grey','purple','yellow'], alpha = .70)
fig.set_xlabel('Attrition Flag')
fig.set_ylabel('No. of Customers')
fig.legend(['College','Docterate','Graduate','High School','Post-Graduate','Uneducated','Unknown'])
plt.show()

In [ ]:
from collections import Counter
cnt = Counter(att_cus['Education_Level'])
perct = [(i, cnt[i] / len(att_cus['Education_Level']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
att_cus_edu = new.rename(columns={ 0 :'Education_Level', 1:'Percentage'})

In [ ]:
from collections import Counter
cnt = Counter(ext_cus['Education_Level'])
perct = [(i, cnt[i] / len(ext_cus['Education_Level']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
ext_cus_edu = new.rename(columns={ 0 :'Education_Level', 1:'Percentage'})

In [ ]:
specs = [[{'type':'pie'}, {'type':'pie'}]]
fig = make_subplots(rows=1, cols=2, specs=specs,subplot_titles=['Attrited Customer','Existing Customer'])
fig.add_trace(go.Pie(labels=att_cus_edu['Education_Level'], values=att_cus_edu['Percentage'],name='No. of Customers'),1,1)
fig.add_trace(go.Pie(labels=ext_cus_edu['Education_Level'], values=ext_cus_edu['Percentage'],name='No. of Customers'),1,2)
fig=go.Figure(fig)
fig.update_layout(title_text='Education Level Distribution by Attrition')
fig.show()

In [ ]:
att_card=df.groupby(['Attrition_Flag','Card_Category']).Card_Category.count().unstack()
print(att_card)

In [ ]:
fig=att_card.plot(kind='bar',title='Attrition Flag by Card Category',color = ['blue','yellow','grey','purple'], alpha = .70)
fig.set_xlabel('Attrition Flag')
fig.set_ylabel('No. of Customers')
fig.legend(['Blue','Gold','Platinum','Silver'])
plt.show()

In [ ]:
from collections import Counter
cnt = Counter(att_cus['Card_Category'])
perct = [(i, cnt[i] / len(att_cus['Card_Category']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
att_cus_card = new.rename(columns={ 0 :'Card_Category', 1:'Percentage'})

In [ ]:
from collections import Counter
cnt = Counter(ext_cus['Card_Category'])
perct = [(i, cnt[i] / len(ext_cus['Card_Category']) * 100.0) for i in cnt]
new = pd.DataFrame(perct)
ext_cus_card = new.rename(columns={ 0 :'Card_Category', 1:'Percentage'})

In [ ]:
specs = [[{'type':'pie'}, {'type':'pie'}]]
fig = make_subplots(rows=1, cols=2, specs=specs,subplot_titles=['Attrited Customer','Existing Customer'])
fig.add_trace(go.Pie(labels=att_cus_card['Card_Category'], values=att_cus_card['Percentage'],name='No. of Customers'),1,1)
fig.add_trace(go.Pie(labels=ext_cus_card['Card_Category'], values=ext_cus_card['Percentage'],name='No. of Customers'),1,2)
fig=go.Figure(fig)
fig.update_layout(title_text='Card Category Distribution by Attrition')
fig.show()

## PyCaret 

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *
classification = setup(data=df,target='Attrition_Flag',train_size=0.8)

In [ ]:
best=compare_models()

In [ ]:
lgb = create_model('lightgbm')

In [ ]:
plot_model(lgb)

In [ ]:
evaluate_model(lgb)

## Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# Create Label
y=df['Attrition_Flag']

In [ ]:
# Create Attribute
x=df.drop(['Attrition_Flag','CLIENTNUM'],axis=1)

In [ ]:
# Encode Label - 0,1 array
encoder=LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

In [ ]:
import seaborn as sns
sns.pairplot(df, hue='Attrition_Flag')

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
# Encoding
x['Gender']=encoder.fit_transform(x['Gender'])
x['Education_Level']=encoder.fit_transform(x['Education_Level'])
x['Marital_Status']=encoder.fit_transform(x['Marital_Status'])
x['Income_Category']=encoder.fit_transform(x['Income_Category'])
x['Card_Category']=encoder.fit_transform(x['Card_Category'])

In [ ]:
# Standardizing
scaler=StandardScaler()
X=scaler.fit_transform(x)

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,encoded_y,test_size=0.2)

In [ ]:
# Create Model
model=Sequential()

# Input layer Layer
model.add(Dense(32,activation='relu',input_dim=19))

# Hidden Layer
model.add(Dense(16,activation='relu'))

# Outout Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# Fit model
model.fit(X_train,y_train,batch_size=10,epochs=100)

In [ ]:
eval_model=model.evaluate(X_train,y_train)

In [ ]:
eval_model

In [ ]:
y_pred=model.predict(X_test)
y_pred =(y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)